In [11]:
import pygame
import math
from Objects import Walls, Wall, Floors, Floor, Sources, ResourceSource, CounterTops, CounterTop, CBoards, CBoard, Fryers, Fryer
from constants import *
from floor_plan import floor_plan_matrix
pygame.init()

(5, 0)

In [12]:
Players = pygame.sprite.Group()
class Player(Object):
    def __init__(self, position, player_graphic, controls):
        super().__init__(position, player_graphic)
        self.controls = controls
        self.hands = None
        self.speed = 4
        self.action = None
        self.HAND_LENGTH = SIZE * math.sqrt(2) - 5 #Slightly smaller than allowing you to get it with a 45 degree angle
        self.action_cooldown = 0
        Players.add(self)
        
    
    def update(self, keys):
        #Actions - grabbing a resource, putting down a reource; only one action can be done in one update
        self.action_cooldown -= 1
        if(keys[self.controls['ACTION']] & isinstance(self.action, CBoard)):
           pass
        else:
           self.action = None 
            
        if (keys[self.controls['ACTION']] & (self.action_cooldown <= 0)):
            action_made = False
            for source in Sources:
                if(not action_made):
                    action_made = action_made or self.grab_resource(source)
            
            for countertop in CounterTops:
                if (not action_made):
                    action_made = action_made or self.put_down_resource(countertop)
                    
                if(isinstance(countertop, CBoard)):
                    if (not action_made):
                        action_made = action_made or self.chop(countertop)
                
                if(isinstance(countertop, Fryer)):
                    if (not action_made):
                        action_made = action_made or self.fry(countertop)
                
                if (not action_made):
                    action_made = action_made or self.take_resource_from_table(countertop)
                    
                    
            if(action_made):
                self.action_cooldown = 20
                    
        
        #Move up down, left, or right
        self.moved = False
        prex, prey = self.rect.x, self.rect.y
        if keys[self.controls['UP']]:
            self.rect.y -= self.speed
            self.moved = True
            
        if keys[self.controls['DOWN']]:
            self.rect.y += self.speed
            self.moved = True
            
        if keys[self.controls['LEFT']]:
            self.rect.x -= self.speed
            self.moved = True
            
        if keys[self.controls['RIGHT']]:
            self.rect.x += self.speed
            self.moved = True
            
        self.last_move = (self.rect.x - prex, self.rect.y - prey)
        if(self.moved):
            self.check_collision()
            
    
    #Get back to the former position if you collided with a wall or a player
    def bounce_back(self):
        self.rect.x, self.rect.y = self.rect.x - self.last_move[0], self.rect.y - self.last_move[1]
        
        
    #Check for collisions with Walls and the other players
    def check_collision(self):
        collisions = pygame.sprite.spritecollide(self, Walls, False)
        if(collisions):
            self.bounce_back()
            
        players = 0
        for player in Players:
            if(self.rect.colliderect(player)):
                players += 1
                
        assert(players > 0)
        if(players > 1):
            self.bounce_back()
    
    #See if the resource source is close enough and if so, grab a resource
    def grab_resource(self, source):
        if ((self.check_distance(source) < self.HAND_LENGTH) & (self.hands == None)):
            source.give_resource(self)
            return True
        
        return False
            
    #See if the countertop is close enough and if so, put down a resource   
    def put_down_resource(self, table):
        #Allow only scenarios where the table is within reach and hands are not empty        
        if(self.hands == None or self.check_distance(table) >= self.HAND_LENGTH):
            return False
        
        #Adding on an empty table
        if (table.resource == None):
            condition = True
            #To put it on a fryer, it must be chopped
            if(isinstance(table, Fryer) and not self.hands.chopped):
                condition = False
                
            if(isinstance(self.hands, Plate) and (isinstance(table, Fryer) or isinstance(table, CBoard))):
                condition = False
            
            if(condition):
                table.put_resource(self.hands)
                self.hands = None
                return True
            return False
        
        #Adding on a plate
        elif (isinstance(table.resource, Plate) and not isinstance(self.hands, Plate) and self.hands.fried):
            table.resource.add_ingredient(self.hands)
            self.hands = None
            return True

        return False

    def take_resource_from_table(self, table):
        if ((self.check_distance(table) < self.HAND_LENGTH) & (self.hands == None) & (table.resource != None)):
            condition = True
            if(isinstance(table, CBoard)):
                condition = table.resource.chopped
                
            if(isinstance(table, Fryer)):
                condition = table.resource.fried
                
            if(condition):
                self.hands = table.resource
                table.resource.place = self
                table.remove_resource()
                return True
            else:
                return False
        
        return False

    def chop(self, CB):
        if ((self.check_distance(CB) < self.HAND_LENGTH) & (CB.resource != None) & (self.hands == None)):
            if(CB.resource.chopped):
                return False
            CB.start_chopping(self)
            self.action = CB
            return True
        return False
    
    def fry(self, Fryer):
        if ((self.check_distance(Fryer) < self.HAND_LENGTH) & (Fryer.resource != None) & (self.hands == None)):
            if(Fryer.resource.fried or Fryer.frying):
                return False
            
            Fryer.start_frying()
            return True
        return False
        
    


NameError: name 'Object' is not defined

In [ ]:
class ResourceGroup(pygame.sprite.Group):
    #Overriding draw because of a specific nature of resources being drawn
    def draw(self, surface):
        for sprite in self.sprites():
            sprite.determine_position()
        super().draw(surface)


Resources = ResourceGroup()
class Resource(pygame.sprite.Sprite):
    def __init__(self, graphics, place):
        self.icon = graphics["icon"]
        self.plain_graphic = graphics["plain"]
        self.chopped_graphic = graphics["chopped"]
        self.chopped_icon = graphics["chopped_icon"]
        self.plated_icon = graphics["plated_icon"]
        self.chopped = False
        self.fried = False
        self.place = place
        self.position = None
        self.image = None
        self.rect = None
        super().__init__()
        Resources.add(self)
        
        
    def determine_position(self):
        if(isinstance(self.place, Player)):
            player = self.place
            self.position = (player.rect.center[0] + 20, player.rect.center[1] - 20)
            self.image = self.icon
            if(self.chopped):
                self.image = self.chopped_icon
            self.rect = self.image.get_rect(center = self.position)
            
        if(isinstance(self.place, CounterTop)):
            table = self.place
            self.position = table.rect.center
            self.image = self.plain_graphic
            if(self.chopped):
                self.image = self.chopped_graphic
            self.rect = self.image.get_rect(center = self.position)
        
        if(isinstance(self.place, Plate)):
            pass #Its position is determined when the position of the place is determined - see below:
        
        #Displaying subicons of ingredients in a dish
        if(isinstance(self, Plate) and len(self.dish) > 0):
            y_gap = 20
            x_gap = 10
            rows_no = math.ceil(len(self.dish)/2)
            for i in range (len(self.dish)):
                ingredient = self.dish[i]
                ingredient.image = ingredient.plated_icon
                #This many rows
                row = i//2
                y = self.position[1] - rows_no * y_gap + row * y_gap
                x = self.position[0]
                #If it's the left element and is not the last one, it goes to the left. If it is the last one and is the left element, it should stay in the middle
                if ((i % 2 == 0) and (i < len(self.dish) - 1)):
                    x -= x_gap
                
                if (i % 2 == 1):
                    x += x_gap
                
                ingredient.rect = ingredient.image.get_rect(center = (x, y))
            
            
    def chop(self):
        self.chopped = True
        
    def fry(self):
        self.fried = True

class Fish(Resource):
    def __init__(self, place):
        super().__init__(FISH_GRAPHICS, place)

class Potato(Resource):
    def __init__(self, place):
        super().__init__(POTATO_GRAPHICS, place)

class Plate(Resource):
    def __init__(self, place):
        super().__init__(PLATE_GRAPHICS, place)
        self.dish = []
        self.dish_dict = {Fish: 0, Potato: 0}
    
    def add_ingredient(self, ingredient):
        self.dish.append(ingredient)
        ingredient.place = self
        if isinstance(ingredient, Fish):
            self.dish_dict[Fish] += 1
            
        elif isinstance(ingredient, Potato):
            self.dish_dict[Potato] += 1

In [ ]:
fish_x, fish_y = (8, 1)
floor_plan_matrix[fish_y][fish_x] = 3
FishCrate = ResourceSource((START_X+SIZE/2 + fish_x*SIZE, START_Y+SIZE/2 + fish_y*SIZE), Fish, FISH_CRATE_ICON)

potato_x, potato_y = (COLS-2, 5)
floor_plan_matrix[potato_y][potato_x] = 3
PotatoCrate = ResourceSource((START_X+SIZE/2 + potato_x*SIZE, START_Y+SIZE/2 + potato_y*SIZE), Potato, POTATO_CRATE_ICON)

plate_x, plate_y = (WALL_COLUMN+1, 1)
floor_plan_matrix[plate_y][plate_x] = 3
PotatoCrate = ResourceSource((START_X+SIZE/2 + plate_x*SIZE, START_Y+SIZE/2 + plate_y*SIZE), Plate, PLATE_CRATE)


CB1_x, CB1_y = fish_x-1, fish_y
floor_plan_matrix[CB1_y][CB1_x] = 3
CBoard1 = CBoard((START_X+SIZE/2 + CB1_x*SIZE, START_Y+SIZE/2 + CB1_y*SIZE))


CB2_x, CB2_y = potato_x, potato_y+1
floor_plan_matrix[CB2_y][CB2_x] = 3
CBoard2 = CBoard((START_X+SIZE/2 + CB2_x*SIZE, START_Y+SIZE/2 + CB2_y*SIZE), None, CB_R_ICON)


Fryer1_x, Fryer1_y = CB1_x-1, CB1_y
floor_plan_matrix[Fryer1_y][Fryer1_x] = 3
Fryer1 = Fryer((START_X+SIZE/2 + Fryer1_x*SIZE, START_Y+SIZE/2 + Fryer1_y*SIZE))

Fryer2_x, Fryer2_y = CB2_x, CB2_y+1
floor_plan_matrix[Fryer2_y][Fryer2_x] = 3
Fryer2 = Fryer((START_X+SIZE/2 + Fryer2_x*SIZE, START_Y+SIZE/2 + Fryer2_y*SIZE), None, FRYER_R_ICON)

CBelt1_x, CBelt1_y = plate_x+1, plate_y
floor_plan_matrix[CBelt1_y][CBelt1_x] = 3
CBelt1 = CBelt((START_X+SIZE/2 + CBelt1_x*SIZE, START_Y+SIZE/2 + CBelt1_y*SIZE))

CBelt2_x, CBelt2_y = CBelt1_x+1, CBelt1_y
floor_plan_matrix[CBelt2_y][CBelt2_x] = 3
CBelt2 = CBelt((START_X+SIZE/2 + CBelt2_x*SIZE, START_Y+SIZE/2 + CBelt2_y*SIZE))

CBelt3_x, CBelt3_y = CBelt1_x, CBelt1_y-1
floor_plan_matrix[CBelt3_y][CBelt3_x] = 3
CBelt3 = CBelt((START_X+SIZE/2 + CBelt3_x*SIZE, START_Y+SIZE/2 + CBelt3_y*SIZE))

CBelt4_x, CBelt4_y = CBelt1_x+1, CBelt1_y-1
floor_plan_matrix[CBelt4_y][CBelt4_x] = 3
CBelt4 = CBelt((START_X+SIZE/2 + CBelt4_x*SIZE, START_Y+SIZE/2 + CBelt4_y*SIZE))


Trash_x, Trash_y = CBelt2_x+1, CBelt2_y
floor_plan_matrix[Trash_y][Trash_x] = 3
Trash = TrashCan((START_X+SIZE/2 + Trash_x*SIZE, START_Y+SIZE/2 + Trash_y*SIZE))

In [ ]:
CBelts = pygame.sprite.Group()
class CBelt(CounterTop):
    def __init__(self, position, graphic = WAITER_POINT):
        super().__init__(position, graphic)
        CBelt.add(self)
        
    def put_resource(self, resource):
        #Can only serve food on a plate
        if(not isinstance(resource, Plate)):
            return 0
        
        if(isinstance(resource, Plate)):
            if(Menu.serve_dish(resource)):
                [x.kill() for x in resource.dish]
                resource.kill()

class TrashCan(CBelt):
    def __init__(self, position, graphic = TRASH_CAN):
        super().__init__(position, graphic)
    
    #Overriding - same as CBelt, but without registering the dish
    def put_resource(self, resource):
        if(isinstance(resource, Plate)):
            [x.kill() for x in resource.dish]
            
        resource.kill()



In [ ]:
obj_types = [[Floor, Floors], [Wall, Walls], [CounterTop, CounterTops]]
for i in range (ROWS):
    for j in range (COLS):
        if(floor_plan_matrix[i][j] < 3):
            obj_class, obj_sprite = obj_types[floor_plan_matrix[i][j]]
            object = obj_class([START_X+SIZE/2 + j*SIZE, START_Y+SIZE/2 + i*SIZE])
            obj_sprite.add(object)

player1_controls = {
    "UP": pygame.K_w,
    "DOWN": pygame.K_s,
    "LEFT": pygame.K_a,
    "RIGHT": pygame.K_d,
    "ACTION": pygame.K_e
}

player2_controls = {
    "UP": pygame.K_UP,
    "DOWN": pygame.K_DOWN,
    "LEFT": pygame.K_LEFT,
    "RIGHT": pygame.K_RIGHT,
    "ACTION": pygame.K_SPACE
}

player1_start = (START_X + 7*SIZE, START_Y + 3*SIZE)
Player1 = Player(player1_start, PLAYER1_GRAPHIC, player1_controls)
Players.add(Player1)

player2_start = (END_X - 3*SIZE, START_Y + 7*SIZE)
Player2 = Player(player2_start, PLAYER2_GRAPHIC, player2_controls)
Players.add(Player2)

In [ ]:
#Menu is a queue
menu = []
import random

MARGIN = 20
DISH_WIDTH = 75
DISH_HEIGHT = 100
MENU_X = START_X
MENU_Y = START_Y - DISH_HEIGHT - 3*MARGIN



class Dish(pygame.sprite.Sprite):
    def __init__(self, name, ingredients, ingredients_dict, score, graphic, order):
        super().__init__()
        self.name = name
        self.ingredients = ingredients
        self.ingredients_dict = ingredients_dict
        self.score = score
        self.graphic = graphic #TO-DO: GET THE FISH AND CHIPS TOO
        self.image = pygame.Surface((DISH_WIDTH, DISH_HEIGHT))
        self.order_in_queue = order
        self.rect = self.image.get_rect(topleft=(MENU_X + (MARGIN+DISH_WIDTH) * self.order_in_queue, MENU_Y))

    def update(self):
        #White rectangle, black border
        self.rect.x = MENU_X + (MARGIN+DISH_WIDTH) * self.order_in_queue
        pygame.draw.rect(screen, WHITE, self.rect.get_rect())
        pygame.draw.rect(screen, BLACK, self.rect.get_rect(), 2)
                
        #Write the name of the 
        font = pygame.font.SysFont(None, 36)
        text = font.render(self.name, True, (0, 0, 0))
        text_rect = text.get_rect(center=(DISH_WIDTH // 2, DISH_HEIGHT // 4))
        self.image.blit(text, text_rect)

        img = self.graphic
        img = pygame.transform.scale(img, (DISH_WIDTH // 2, DISH_HEIGHT // 2))
        img_rect = img.get_rect(center=(DISH_WIDTH // 2, 3 * DISH_HEIGHT // 4))
        self.image.blit(img, img_rect)
        
        
class Menu():
    def __init__(self):
        self.queue = []
        self.max_dishes = 5
        self.frequency = 15 #Once every X seconds on average, there will be a new dish
        self.game_score = 0
    
    def update(self):
        #Add a dish 
        #The number of dishes should be capped
        if(len(self.queue) < self.max_dishes):
            decider = random.random()
            probability = self.frequency/1000*60
            if (decider < probability):
                self.generate_new_dish()

        for i in range (len(self.queue)):
            self.queue[i].order_in_queue = i
            self.queue[i].update()
            
            
    def generate_new_dish(self, prob_chips = 0.5):
        decider = random.random()
        if(decider < prob_chips):
            #Add fish and chips
            self.queue.append(Dish("Fish and Chips", [Fish, Potato], {Fish: 1, Potato: 1}, 100, FISH_CHOPPED_ICON, len(self.queue)))
            
        else:
            #Add just fish
            self.queue.append(Dish("Fish", [Fish], {Fish: 1, Potato: 0}, 50, FISH_CHOPPED_ICON, len(self.queue)))
            
    def serve_dish(self, plate):
        for i in range (len(self.queue)):
            order = self.queue[i]
            if (order.ingredients_dict[Fish] == plate.dish_dict[Fish]) and (order.ingredients_dict[Potato] == plate.dish_dict[Potato]):
                self.game_score += order.score
                self.queue.pop(i)
                return True
            
        return False
                    
Menu = Menu()



In [ ]:
def draw():
    screen.fill((255, 255, 255)) 
    Floors.draw(screen)
    Walls.draw(screen)
    Resources.draw(screen)
    Players.draw(screen)

In [ ]:
import numpy as np

def get_state():
    pixel_array = pygame.surfarray.array3d(screen)
    # Transpose to (height, width, channels)
    pixel_array = np.transpose(pixel_array, (1, 0, 2))
    
    #Normalize so that pixel values are from 0 to 1, not 0 to 255 as in RGB
    pixel_array = pixel_array / 255.0
    pixel_array = np.expand_dims(pixel_array, axis=0)

In [ ]:
time_per_game = int(input("Please enter the length of the game, in seconds (default: 180)"))
start_time = pygame.time.get_ticks()

def update():
    elapsed_time = (pygame.time.get_ticks() - start_time) / 1000
    if(elapsed_time < time_per_game):
        Players.update(pygame.key.get_pressed())
        CBoards.update()
        Fryers.update()
        Menu.update()

    else:
        #Game finished; display a large "GAME OVER" sign over a frozen frame
        font = pygame.font.SysFont("comicsansms", 100)
        game_over_surface = font.render("GAME OVER", True, (255, 0, 0))
        game_over_rect = game_over_surface.get_rect(center=((START_X+END_X)//2, (START_Y + END_Y)//2))
        screen.blit(game_over_surface, game_over_rect)

In [ ]:
from model.py import model

input_size = 
output_size = 
Misha = model()


running = True
clock = pygame.time.Clock()
tick = 0
game_score = 0

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            break

    if(not running):
        break
    
    draw()
    update()
    
    pixel_input = get_state()
    
    pygame.display.flip()
    clock.tick(60)
        
pygame.display.quit()
pygame.quit()
exit()

AttributeError: 'pygame.rect.Rect' object has no attribute 'get_rect'